In [35]:

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [36]:
# Specify and load datastructure specifications
filename='ForMia_netcdf_specs.xlsx'
directory=''
df_data = pd.read_excel(directory+filename,sheet_name='coords')
unit_dict = dict(zip(df_data['var_name'],df_data['units']))
df_coord = pd.read_excel(directory + filename, sheet_name='coords')
unit_dict.update(dict(zip(df_coord['var_name'],df_coord['units'])))
num_dict = dict(zip(df_data['var_name'],df_data['number']))
encoding_dict = {}
encoding_val = {'zlib':True,'_FillValue':0.0}

# Specify specific data structure creation
# options 'Final', 'end-point', 'across-hr', 'across-ls'
sheet = 'across-ls'

In [37]:
# Setup Datastructure Variables (coords
grid =  xr.Dataset()
filename_netCDF = '__J9GOH9T31G_singlePoint.nc'
sample = xr.open_dataset(filename_netCDF, decode_cf=False)
ori = ['lat', 'lon', 'ls', 'hr']
ori_dict = dict(zip(ori,[np.arange(-90,91,10), np.arange(-180,181,10), np.arange(0,360,15), np.arange(0,25,2)]))

for vari in df_coord['var_name']:
    if vari in ori:
        grid[vari] = ori_dict[vari]
    else:
        grid[vari] = sample.coords[vari].values
    grid.coords[vari].attrs['units'] = unit_dict[vari]
    encoding_dict[vari] = encoding_val

In [38]:
# Create the specific datastructure
df_data = pd.read_excel(directory+filename,sheet_name=sheet)
unit_dict = dict(zip(df_data['var_name'],df_data['units']))
coord_dict = dict(zip(df_data['var_name'],df_data['dims']))
for vari in df_data['var_name']:
    coords = coord_dict[vari].split(',')
    zi = [len(grid.coords[i]) for i in coords]
    grid[vari] = (tuple(coords), np.zeros(zi,dtype=np.uint8))
    grid[vari].attrs['units'] = unit_dict[vari]
    encoding_dict[vari] = encoding_val

In [39]:
# Save as netCDF file.
grid.to_netcdf(sheet+ '.nc',encoding=encoding_dict)